In [ ]:
!pip install nest_asyncio
!pip install python-telegram-bot --upgrade
!pip install APScheduler --upgrade
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
import nest_asyncio
import asyncio
import datetime
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes

# Разрешаем повторное использование событийного цикла
nest_asyncio.apply()

# Загрузка модели
model = joblib.load("catboost_model.pkl")
print("Модель успешно загружена.")

# Токен бота
BOT_TOKEN = '7719173614:AAFOxYs-5Gjqb8tTkrRCHhPkzEllgC0p8vs'  # Укажи свой токен

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text("Привет! Я бот для прогнозирования цен на арматуру.\n"
                                    "Доступные команды:\n"
                                    "/start - Начало работы\n"
                                    "/help - Список команд\n"
                                    "/predict <цена> - Прогноз объема закупки\n"
                                    "/forecast <цена> - Прогноз цен на 6 недель вперед")

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text("Доступные команды:\n"
                                    "/start - Начало работы\n"
                                    "/help - Список команд\n"
                                    "/predict <цена> - Прогноз объема закупки\n"
                                    "/forecast <цена> - Прогноз цен на 6 недель вперед")

async def predict(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
        if not context.args:
            await update.message.reply_text("Введите цену после команды. Например: /predict 75000")
            return

        current_price = float(context.args[0])
        if current_price <= 0 or current_price > 1e7:
            await update.message.reply_text("Ошибка: Введите реалистичную цену. Например: /predict 75000")
            return

        # Заглушка для предсказания (замени на свою логику)
        predicted_weeks = max(1, min(6, int(6 - (current_price / 10000))))
        await update.message.reply_text(f"Рекомендуемый объем закупки: на {predicted_weeks} недель.")

    except ValueError:
        await update.message.reply_text("Ошибка: Введите корректное число. Например: /predict 75000")
    except Exception as e:
        await update.message.reply_text(f"Ошибка при предсказании: {str(e)}")

async def forecast(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
        if not context.args:
            await update.message.reply_text("Введите текущую цену. Например: /forecast 75000")
            return

        current_price = float(context.args[0])
        if current_price <= 0 or current_price > 1e7:
            await update.message.reply_text("Ошибка: Введите реалистичную цену. Например: /forecast 75000")
            return

        future_weeks = 6
        price_forecast = []

        # Начальные данные для предсказания
        last_X = pd.DataFrame({
            'istest': [1],
            'Price_source': [current_price],
            'Price_Diff': [0]  # Начальная разница (можно уточнить)
        })

        for _ in range(future_weeks):
            pred_price = model.predict(last_X)[0]
            price_forecast.append(pred_price)

            # Обновляем входные данные
            last_X['Price_Diff'] = pred_price - last_X['Price_source']
            last_X['Price_source'] = pred_price

        # Добавляем начальную цену в начало прогноза
        price_forecast = [current_price] + price_forecast
        future_weeks = len(price_forecast)

        # Создание улучшенного графика
        plt.figure(figsize=(12, 7))
        plt.plot(range(0, future_weeks), price_forecast, marker='o', linestyle='-', color='forestgreen', linewidth=2, label='Прогнозируемая цена')

        # Добавляем подписи к точкам на графике
        for i, price in enumerate(price_forecast):
            plt.text(i, price, f"{price:.2f}", ha='center', va='bottom', fontsize=10, color='darkgreen', fontweight='bold')

        # Настройки графика
        plt.title("Прогноз цен на арматуру на 6 недель", fontsize=16, fontweight='bold', color='darkgreen')
        plt.xlabel("Недели", fontsize=14, color='darkgreen')
        plt.ylabel("Цена", fontsize=14, color='darkgreen')
        plt.legend(loc='upper left', fontsize=12)
        plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

        # Улучшаем внешний вид
        plt.xticks(range(0, future_weeks))
        plt.tight_layout()

        # Сохранение графика
        plot_filename = "forecast_plot.png"
        plt.savefig(plot_filename, dpi=300)
        plt.close()


        # Отправка результата
        await update.message.reply_text(f"Прогнозируемые цены на 6 недель: {price_forecast}")
        with open(plot_filename, "rb") as plot_file:
            await update.message.reply_photo(photo=plot_file)

    except ValueError:
        await update.message.reply_text("Ошибка: Введите корректную цену. Например: /forecast 75000")
    except Exception as e:
        await update.message.reply_text(f"Ошибка при предсказании: {str(e)}")

# Запуск бота
if __name__ == "__main__":
    application = ApplicationBuilder().token(BOT_TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("predict", predict))
    application.add_handler(CommandHandler("forecast", forecast))
    application.run_polling()


Модель успешно загружена.


RuntimeError: Cannot close a running event loop

In [ ]:
ыimport nest_asyncio
import asyncio
import datetime
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import numpy as np  # Для генерации случайных данных

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes

# Разрешаем повторное использование событийного цикла (для Jupyter Notebook и Colab)
nest_asyncio.apply()

# Загрузка модели из файла
model = joblib.load('/content/catboost_optimized.pkl')
print("Модель успешно загружена.")

# Токен бота
BOT_TOKEN = '7719173614:AAFOxYs-5Gjqb8tTkrRCHhPkzEllgC0p8vs'  # Замените на ваш токен

# Стартовое сообщение
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        "Привет! Я бот для прогнозирования объема закупки арматуры.\n"
        "Доступные команды:\n"
        "/start - Начало работы с ботом\n"
        "/help - Информация о командах\n"
        "/predict <цена> - Прогноз объема закупки на основе цены\n"
    )

# Сообщение с информацией о командах
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        "Доступные команды:\n"
        "/start - Начало работы с ботом\n"
        "/help - Информация о командах\n"
        "/predict <цена> - Прогноз объема закупки на основе цены\n"
    )

async def predict(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
        # Проверка на наличие аргументов
        if not context.args:
            await update.message.reply_text(
                "Пожалуйста, укажите цену после команды. Например: /predict 75000"
            )
            return

        # Получаем цену из аргументов
        current_price = float(context.args[0])
        current_date = datetime.datetime.now()
        week_number = current_date.isocalendar()[1]  # Номер недели в году
        year_number = current_date.year

        # Генерация случайных лагов (замените на фактические данные из истории)
        lag_values = [current_price * (1 + np.random.uniform(-0.1, 0.1)) for _ in range(6)]  # Случайные колебания цены

        # Преобразуем дату в числовой формат (количество дней с начала эпохи)
        epoch = datetime.datetime(1970, 1, 1)
        days_since_epoch = (current_date - epoch).days

        # Подготовка данных для модели
        input_data = pd.DataFrame([[
            days_since_epoch,  # dt (в числовом формате)
            0,                 # istest
            week_number,       # Week
            year_number,       # Year
            *lag_values        # Lag_1 - Lag_6
        ]], columns=['dt', 'istest', 'Week', 'Year', 'Lag_1', 'Lag_2', 'Lag_3', 'Lag_4', 'Lag_5', 'Lag_6'])

        # Логирование входных данных
        print("Входные данные для модели:")
        print(input_data)

        # Предсказание
        predicted_weeks = model.predict(input_data)[0]
        print(f"Предсказание модели: {predicted_weeks}")

        # Создаем график
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, 7), lag_values, marker='o', label='Исторические данные (лаги)')
        plt.axhline(y=current_price, color='r', linestyle='--', label='Текущая цена')
        plt.title(f"Прогноз объема закупки на {int(predicted_weeks)} недель")
        plt.xlabel("Недели")
        plt.ylabel("Цена")
        plt.legend()
        plt.grid(True)

        # Сохраняем график в файл
        plot_filename = "plot.png"
        plt.savefig(plot_filename)
        plt.close()  # Закрываем график, чтобы освободить память

        # Отправляем текстовое сообщение с результатом
        await update.message.reply_text(f"Рекомендуемый объем закупки: на {int(predicted_weeks)} недель.")

        # Отправляем график
        with open(plot_filename, "rb") as plot_file:
            await update.message.reply_photo(photo=plot_file)

    except ValueError:
        await update.message.reply_text("Ошибка: Укажите корректное число после команды. Например: /predict 75000")
    except Exception as e:
        await update.message.reply_text(f"Ошибка при предсказании: {str(e)}")

# Главная функция запуска бота
def main() -> None:
    application = ApplicationBuilder().token(BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("predict", predict))

    # Запуск бота
    application.run_polling()

# Стартуем бота
if __name__ == "__main__":
    main()

Модель успешно загружена.


ERROR:telegram.ext:Network Retry Loop (Bootstrap Initialize Application): Invalid token. Aborting retry loop.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 835, in initialize
    await self.get_me()
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 1969, in get_me
    return await super().get_me(
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 967, in get_me
    result = await self._post(
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 691, in _post
    return await self._do_post(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 362, in _do_post
    return await super()._do_post(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 720, in _do_post
    result = await reques

RuntimeError: Cannot close a running event loop

In [ ]:
# import nest_asyncio
# import asyncio
# from telegram import Update
# from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes

# # Разрешаем повторное использование событийного цикла (для Jupyter Notebook и Colab)
# nest_asyncio.apply()

# BOT_TOKEN = '7888904212:AAG4ZkvWMLjBuZjxGzZOe3AzdmpmpB8uFg0'  # Замените на ваш токен

# async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
#     """Отправляет приветственное сообщение при команде /start."""
#     await update.message.reply_text('Привет! Я бот, написанный ИСП-21 для интенсива!\n'
#             "Доступные команды:\n"
#         "/start - Начало работы с ботом\n"
#         "/help - Информация о командах\n"
#         "/info - Информация\n")

# async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
#     """Отправляет сообщение с информацией о командах."""
#     await update.message.reply_text(
#         "Доступные команды:\n"
#         "/start - Начало работы с ботом\n"
#         "/help - Информация о командах\n"
#         "/info - Информация\n"
#     )

# async def info(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
#     """Повторяет сообщение пользователя."""
#     await update.message.reply_text("Это бот для интенсива с предсказанием цен на арматуру")

# def main() -> None:
#     """Запускает бота."""
#     application = ApplicationBuilder().token(BOT_TOKEN).build()

#     application.add_handler(CommandHandler("start", start))
#     application.add_handler(CommandHandler("help", help_command))
#     application.add_handler(CommandHandler("info", info))

#     # Запускаем бота
#     application.run_polling()

# if __name__ == "__main__":
#     main()